### EVALUACIÓN API NATALIA BARQUÍN

----------------

In [1]:
import requests
import pandas as pd
import numpy as np
import sidetable

from geopy.geocoders import Nominatim

import mysql.connector

import os
from dotenv import load_dotenv
load_dotenv()

True

1. Utilizando la API extraed toda la información que podáis de ella.

In [2]:
def extraer_df_api():

    """
    Crea una lista con los 3 países que queremos estudiar y realiza la llamada a la API y extracción de datos de la misma para cada uno de los países.
    Después crea un dataframe con los datos de cada país y los unifica en un único dataframe.

    Args:
        No recibe.
        
    Returns:
        df: dataframe unido con los datos de los 3 países.
    """

    lista_paises = ['United States', 'Canada' , 'Argentina']

    df_unido = pd.DataFrame()

    for pais in lista_paises:

        url = f'http://universities.hipolabs.com/search?country={pais}'

        response = requests.get(url=url)

        response.status_code

        response.reason
        
        df = pd.json_normalize(response.json())

        df_unido = pd.concat([df_unido, df], ignore_index = True)
    
    return df_unido

In [3]:
df_universidades = extraer_df_api()
df_universidades.head()

,state-province,country,domains,web_pages,alpha_two_code,name
0,None,United States,[marywood.edu],[http://www.marywood.edu],US,Marywood University
1,None,United States,[lindenwood.edu],[http://www.lindenwood.edu/],US,Lindenwood University
2,None,United States,[sullivan.edu],[https://sullivan.edu/],US,Sullivan University
3,None,United States,[fscj.edu],[https://www.fscj.edu/],US,Florida State College at Jacksonville
4,None,United States,[xavier.edu],[https://www.xavier.edu/],US,Xavier University


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [4]:
def homogeneizar_columnas(df, col):

    """Reemplaza los guiones de los nombres de las columnas del dataframe por barra baja.
        Elimina columnas redundantes.
    Args:
        df: dataframe original.
        col: columna redundante
    Returns:
        df: dataframe modificado.
    """
    
    df.rename(columns = {col: col.replace('-', '_') for col in df.columns}, inplace = True)
    df.drop(col, axis = 1, inplace = True)

    return df


In [5]:
homogeneizar_columnas(df_universidades, 'domains').head()

,state_province,country,web_pages,alpha_two_code,name
0,None,United States,[http://www.marywood.edu],US,Marywood University
1,None,United States,[http://www.lindenwood.edu/],US,Lindenwood University
2,None,United States,[https://sullivan.edu/],US,Sullivan University
3,None,United States,[https://www.fscj.edu/],US,Florida State College at Jacksonville
4,None,United States,[https://www.xavier.edu/],US,Xavier University


*Hemos observado que algunos registros de la columna name contienen comillas que posteriormente dificultarán la inserción de datos, las eliminaremos.*

In [6]:
def limpiar_comillas(df, col):

    """Elimina las comillas de los registros de la columna que se le indique.
    Args:
        df: dataframe original.
        col: columna sobre la que queremos eliminar las comillas.
    Returns:
        df: dataframe con la columna sin comillas.
    """
            
    return df[col].str.replace('"', '').str.replace("'", "")

In [7]:
df_universidades['name'] = limpiar_comillas(df_universidades, 'name')

3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. 

In [8]:
def aplicar_explode(df, col):

    """Aplica método explode a columnas con más de un valor por registro.
    Args:
        df: dataframe original.
        col: col a aplicar explode, parámetro por defecto.
    Returns:
        df: dataframe modificado.
    """
    
    return df.explode(col, ignore_index= True)

In [9]:
df_universidades = aplicar_explode(df_universidades, 'web_pages')

In [10]:
df_universidades[df_universidades['name'] == 'Cégep de Saint-Jérôme']

,state_province,country,web_pages,alpha_two_code,name
2278,Quebec,Canada,https://www.cstj.qc.ca,CA,Cégep de Saint-Jérôme
2279,Quebec,Canada,https://ccmt.cstj.qc.ca,CA,Cégep de Saint-Jérôme
2280,Quebec,Canada,https://ccml.cstj.qc.ca,CA,Cégep de Saint-Jérôme


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [11]:
def eliminar_duplicados(df, col):

    """Elimina las filas duplicadas en base a una columna.

    Args:
        df: dataframe.
        col: columna sobre la que buscar duplicados.

    Returns:
        df: dataframe sin las filas duplicadas.
    """

    if df.duplicated([col]).sum() > 0:
        print(f'Tenemos {df.duplicated([col]).sum()} duplicados en la columna "{col}", los eliminaremos.')
        return df.drop_duplicates(subset = col, inplace = True, ignore_index = True)
    
    else:
        print(f'No tenemos duplicados en la columna {col}.')

In [12]:
eliminar_duplicados(df_universidades, 'name')

Tenemos 26 duplicados en la columna "name", los eliminaremos.


5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [13]:
df_universidades['state_province'].unique()

array([None, 'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX',
       'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA', 'Illinois',
       'New Hampshire', 'North Carolina', 'South Carolina', 'Virginia',
       'Washington', 'Missouri', 'California', 'NY', 'ND', 'MI',
       'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine', 'Quebec',
       'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

In [14]:
def convertir_a_nan(valor):
    """convierte los valores None a np.nan.

    Args:
        valor (str): valor None

    Returns:
        valor (np.nan): valor np.nan
    """

    if valor == None:
        return np.nan
    else:
        return valor

In [15]:
#aplicamos un apply para que modifique todos los registros de la columna
df_universidades['state_province'] = (df_universidades['state_province'].apply(convertir_a_nan))

In [16]:
df_universidades.head()

,state_province,country,web_pages,alpha_two_code,name
0,NaN,United States,http://www.marywood.edu,US,Marywood University
1,NaN,United States,http://www.lindenwood.edu/,US,Lindenwood University
2,NaN,United States,https://sullivan.edu/,US,Sullivan University
3,NaN,United States,https://www.fscj.edu/,US,Florida State College at Jacksonville
4,NaN,United States,https://www.xavier.edu/,US,Xavier University


6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [17]:
df_universidades.stb.missing()

,missing,total,percent
state_province,2204,2492,88.443018
country,0,2492,0.000000
web_pages,0,2492,0.000000
alpha_two_code,0,2492,0.000000
name,0,2492,0.000000


In [18]:
def imputar_nulos(df, col):

    """convierte los valores nulos al valor de string 'Unknow'.

    Args:
        df (df): dataframe.
        col (col): columna sobre la que queremos imputar los valores nulos.

    Returns:
        df: dataframe con nulos imputados.
    """
    
    return df[col].replace(np.nan, 'Unknow', inplace = True)

In [19]:
imputar_nulos(df_universidades, 'state_province')

In [20]:
df_universidades.stb.missing()

,missing,total,percent
state_province,0,2492,0.0
country,0,2492,0.0
web_pages,0,2492,0.0
alpha_two_code,0,2492,0.0
name,0,2492,0.0


7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso. Para desarrollar este ejercicio deberéis: 

- Sacar los valores únicos de la columna state_province.

In [21]:
df_universidades['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA',
       'TX', 'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA',
       'Illinois', 'New Hampshire', 'North Carolina', 'South Carolina',
       'Virginia', 'Washington', 'Missouri', 'California', 'NY', 'ND',
       'MI', 'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos:

In [22]:
#creamos un diccionario donde las keys son los valores originales y los values los valores por los que los queremos cambiar.
dicc_estados = {
    'NV' : 'Nevada',
    'TX' : 'Texas',
    'IN' : 'Indianapolis',
    'CA' : 'California',
    'VA' : 'Virginia',
    'NY' : 'New York',
    'MI' : 'Michigan', 
    'GA' : 'Georgia', 
    'ND' : 'North Dakota', 
    'New York, NY' : 'New York', 
    'Ciudad Autónoma de Buenos Aires' : 'Buenos Aires'}

In [23]:
#aplicamos un apply con una lambda que utiliza un método de diccionario para modificar estos valores
df_universidades["state_province"] = df_universidades['state_province'].apply(lambda estado: dicc_estados.get(estado, estado))

In [24]:
df_universidades['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán'], dtype=object)

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [25]:
lista_estados = df_universidades['state_province'].unique().tolist()

- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [26]:
def sacar_latitud_longitud_geopy(lista):

    """Usando la librería Geopy, obtiene la latitud y la longitud de una lista de provincias que
    le indiquemos, con ello crea un dataframe con las columnas provincia, latitud y longitud.
    Si la provincia es desconocida, devuelve un valor nulo para latitud y longitud.
    Args:
        lista (list): lista con localizaciones.
    Returns:
        df (df): dataframe de 3 columnas (provincia, latitud y longitud).
    """

    lista_latitud = []
    lista_longitud = []

    for estado in lista:

        if estado != 'Unknow': #si el estado no es desconocido, le pedimos que nos de su latitud y longitud
            geo = Nominatim(user_agent = 'Natalia')
            localizacion = geo.geocode(estado)
            lista_latitud.append(localizacion.latitude)
            lista_longitud.append(localizacion.longitude)
    
        else: # en caso de no tener estado, pedimos que nos devuelva nulos
            lista_latitud.append(np.nan)
            lista_longitud.append(np.nan)

    #creamos un diccionario para crear un df, donde las keys serán las columnas y los values los datos que hemos obtenido con la libreria geopy
    diccionario = {
        'state_province' : lista,
        'latitude' : lista_latitud,
        'longitude' : lista_longitud
        }

    #lo guardamos para tenerlo a mano si la librería falla
    df = pd.DataFrame(diccionario)
    df.to_csv('../data/coordenadas.csv') 

    return df


In [27]:
df_geopy = sacar_latitud_longitud_geopy(lista_estados)

In [28]:
def abrir_latitud_longitud_fichero():

    """Abre el archivo guardado en el anterior método. Este método está pensado para poder trabajar con unos datos
    extraídos previamente de la librería geopy en caso de que en el momento actual la librería no funcione bien.
    Args:
        No recibe.
    Returns:
        df (df): dataframe de 3 columnas guardado.
    """       
    
    return pd.read_csv('../data/coordenadas.csv', index_col = 0)

In [29]:
df_coordenadas = abrir_latitud_longitud_fichero()

- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [32]:
def mergear_dfs(df1, df2, col):

    """Mergea los 2 dataframes creados con esta clase.
    Args:
        df1 (df): primer dataframe a unir.
        df2 (df): segundo dataframe a unir.
        col (df): columna por la que queremos unir los dataframes.
    Returns:
        df (df): dataframe unido.
    """

    return df1.merge(df2, on = col)

In [33]:
df_unido = mergear_dfs(df_universidades, df_coordenadas, 'state_province')

8. Crea una BBDD en mysql que contenga las siguientes tablas:

- Tabla países.

- Tabla universidades.


In [34]:
def crear_bbdd(nombre_bbdd):

    """Usando mysql.connector, crea en MySQL una base de datos.

    Args:
        nombre de la base de datos (str): el nombre que queremos poner a nuestra base de datos.

    Returns:
        No tiene. Crea directamente la base de datos en MySQL.
    """

    conexion = mysql.connector.connect(
                                host="localhost",
                                user="root",
                                password= os.getenv('password'))
    
    mycursor = conexion.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        conexion.close()

    # en caso de que no podamos ejecutar la query pedimos que nos devuelva un error para saber en qué nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        conexion.close()

In [35]:
crear_bbdd('Universidades')

In [36]:
def crear_tabla(nombre_bbdd, query):

    """Usando mysql.connector, crea en MySQL una tabla en una base de datos.

    Args:
        nombre de la base de datos (str): la base de datos donde queremos crear nuestra tabla.
        consulta (str): la consulta con la que creamos la tabla.

    Returns:
        No tiene. Crea directamente la tabla en MySQL.
    """
    
    conexion = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  

    mycursor = conexion.cursor()
    
    try: 
        mycursor.execute(query)
        conexion.close()

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        conexion.close()

In [37]:
tabla_paises = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`paises` (
    `id_estado` INT NOT NULL AUTO_INCREMENT,
    `nombre_pais` VARCHAR(45) NOT NULL,
    `nombre_provincia` VARCHAR(45),
    `latitud` FLOAT,
    `longitud`FLOAT,
    PRIMARY KEY (`id_estado`));
'''

tabla_universidades = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`universidades` (
    `id_universidades` INT NOT NULL AUTO_INCREMENT,
    `nombre_universidad` VARCHAR(100) NOT NULL,
    `pagina_web` VARCHAR(100),
    `paises_id_estado` INT NOT NULL,
    PRIMARY KEY (`id_universidades`),
    CONSTRAINT `fk_universidades_paises`
        FOREIGN KEY (`paises_id_estado`)
        REFERENCES `Universidades`.`paises` (`id_estado`));
ENGINE = InnoDB;
'''

In [38]:
crear_tabla('Universidades', tabla_paises)

In [39]:
crear_tabla('Universidades', tabla_universidades)

BONUS

- Introduce los datos en la BBDD de SQL.

- Crea una clase con todo el código generado en esta evaluación.

In [40]:
df_unido.head(1)

,state_province,country,web_pages,alpha_two_code,name,latitude,longitude
0,Unknow,United States,http://www.marywood.edu,US,Marywood University,NaN,NaN


In [41]:
def insertar_datos(nombre_bbdd, query):
    
    """Usando mysql.connector, inserta datos en una tabla de una base de datos.

    Args:
        nombre de la base de datos (str): la base de datos donde queremos insertar los datos.
        consulta (str): la consulta con la que insertamos los datos.

    Returns:
        No tiene. Inserta los datos directamente en la tabla en MySQL.
    """
    
    conexion = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  

    mycursor = conexion.cursor()

    try: 
        mycursor.execute(query)
        conexion.commit() 
        conexion.close()

    # en caso de que no podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        conexion.close()

In [42]:
def sacar_id_estado(nombre_bbdd, estado, pais):

    """Usando mysql.connector, selecciona el id de un estado de nuestra base de datos.

    Args:
        nombre de la base de datos (str): la base de datos de la que queremos extraer la información.
        estado (str): estado del que queremos sacar el id.
        pais (str): país al que pertenece el estado del que queremos sacar el id.

    Returns:
        id (str): el id del estado.
    """
    
    cnx = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  

    mycursor = cnx.cursor()

    query_sacar_id = f"SELECT id_estado FROM paises WHERE nombre_provincia = '{estado}' AND nombre_pais = '{pais}'"
    
    # puede que el id de la ciudad que estamos intentando insertar no este en nuestra BBDD, de modo que usaremos un try except para evitar errores
    try: 
        mycursor.execute(query_sacar_id)
        id_est = mycursor.fetchall()[0][0]
        
        return id_est
    
    except: 
        return "Lo siento, no tenemos ese estado en la BBDD y por lo tanto no te podemos dar su id. "

In [43]:
for indice, fila in df_unido.iterrows():

        latitud = fila['latitude']
        longitud = fila['longitude']

        if pd.isna(latitud):
                latitud = 'NULL'

        if pd.isna(longitud):
                longitud = 'NULL'

        id_estado = sacar_id_estado('Universidades', fila['state_province'],fila['country'])

        if type(id_estado) is not int:

                insertar_paises = f"""
                        INSERT INTO `Universidades`.`paises` (nombre_pais, nombre_provincia, latitud, longitud) 
                        VALUES ( "{fila['country']}", "{fila['state_province']}", {latitud}, {longitud});
                        """
                insertar_datos('Universidades', insertar_paises)
        else:
                pass

In [44]:
for indice, fila in df_unido.iterrows():

        id_estado = sacar_id_estado('Universidades', fila['state_province'], fila['country'])
        insertar_universidades = f"""
                INSERT INTO `Universidades`.`universidades` (nombre_universidad, pagina_web, paises_id_estado) 
                VALUES ( "{fila['name']}", "{fila['web_pages']}", {id_estado});
                """
        
        insertar_datos('Universidades', insertar_universidades)